In [1]:
from dataloader import create_split_loaders
from model import GenericRNN
import torch

In [25]:
import torch.nn as nn
import copy
config = {'chunk_size':100, 'type_number':93, 'hidden':100, 
          'learning_rate':0.001, 'early_stop':False, 'increase_limit':3, 'epoch_num':1, 'N':50, 'M':100}

def train(config):
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        computing_device = torch.device("cuda")
        extras = {"num_workers": 1, "pin_memory": True}
        print("CUDA is supported")
    else: # Otherwise, train on the CPU
        computing_device = torch.device("cpu")
        extras = False
        print("CUDA NOT supported")
    
    # the size of every chunk
    chunk_size = config['chunk_size']
    # number of types, it is a constant
    type_number = config['type_number']
    # number of features in hidden layer
    hidden = config['hidden']
    # learning rate
    learning_rate = config['learning_rate']
    # whether we use early stop
    early_stop = config['early_stop']
    # after validation loss increase how many times do we stop training
    increase_limit = config['increase_limit']
    # number of epoch
    epoch_num = config['epoch_num']
    # receive train, validation, test data
    train, valid, test, c_to, one_to = create_split_loaders(chunk_size,extras)
    # construct network
    net = GenericRNN(type_number, hidden, type_number)
    net = net.to(computing_device)
    # use cross entropy loss
    criterion = nn.BCELoss()
    # keep tracking of the traininig loss
    training_record = []
    # keep tracking of the validation loss
    validation_record = []
    # Using Adam
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

    last_valid = float('inf')
    best_net = -1
    # after how many batches do we record the training loss
    N = config['N']
    # after how many batches do we examine whether validation loss increase
    M = config['M']
    # store best loss
    best_loss = float('inf')
    inresement = 0
    for epoch in range(epoch_num):
        count = 0
        average_loss = 0
        for minibatch in train:
            count += 1
            if minibatch[0].size()[0] != chunk_size:
                break
            predict_all = torch.zeros(chunk_size, type_number)
            target_all = torch.zeros(chunk_size, type_number)
            optimizer.zero_grad()
            for ii in range(chunk_size):
                train_batch = torch.zeros(1, 1, type_number)
                train_batch[0] = minibatch[0][ii]
                train_batch = train_batch.to(computing_device)
                target = minibatch[1][ii]
                target = target.to(computing_device)
                if ii == 0:
                    predict = net.predict(train_batch)
                else:
                    # teacher forcing
                    teacher = torch.ones(1, 1, type_number)
                    teacher[0] = minibatch[1][ii - 1]
                    teacher = teacher.to(computing_device)
                    predict = net.predict(train_batch, teacher)
                predict_all[ii] = predict
                target_all[ii] = target
            # validation and early stop
            if early_stop:
                if count % M == 0:
                    loss_val = 0
                    count_val = 0
                    for val in valid:
                        count_val += 1
                        if val[0].size()[0] != chunk_size:
                            break
                        predict_valid = torch.zeros(chunk_size, type_number)
                        target_valid = torch.zeros(chunk_size, type_number)
                        for ii in range(chunk_size):
                            valid_batch = torch.zeros(1, 1, type_number)
                            valid_batch[0] = val[0][ii]
                            valid_batch = valid_batch.to(computing_device)
                            target = val[1][ii]
                            target = target.to(computing_device)
                            predict = net.predict(valid_batch)
                            predict_valid[ii] = predict
                            target_valid[ii] = target
                        loss_val += criterion(predict_valid, target_valid)
                    loss_val /= count_val
                    validation_record.append(loss_val.item())
                    print('keep tracking of validation error')
                    print(validation_record)
                    if loss_val > last_valid:
                        increasement += 1
                    else:
                        increasement = 0
                    if loss_val < best_loss:
                        best_loss = loss_val
                        best_net = copy.deepcopy(net)
                    last_valid = loss_val
                    if increasement >= increase_limit:
                        break
            # calculate loss
            loss = criterion(predict_all, target_all)
            loss.backward()
            optimizer.step()
            average_loss += loss.item()
            if count % N == 0:
                training_record.append(average_loss / N)
                average_loss = 0
                print('keep tracking of training error:')
                print(training_record)
#             print(loss.item())
        if early_stop:
            if increasement >= increase_limit:
                break
    if early_stop:
        net = best_net
    return net

In [26]:
train(config)

CUDA is supported
keep tracking of training error:
[0.058622940331697466]
keep tracking of training error:
[0.058622940331697466, 0.05302516132593155]
keep tracking of training error:
[0.058622940331697466, 0.05302516132593155, 0.04993059240281582]
keep tracking of training error:
[0.058622940331697466, 0.05302516132593155, 0.04993059240281582, 0.04812738552689552]
keep tracking of training error:
[0.058622940331697466, 0.05302516132593155, 0.04993059240281582, 0.04812738552689552, 0.04519643031060696]
keep tracking of training error:
[0.058622940331697466, 0.05302516132593155, 0.04993059240281582, 0.04812738552689552, 0.04519643031060696, 0.045326415672898294]
keep tracking of training error:
[0.058622940331697466, 0.05302516132593155, 0.04993059240281582, 0.04812738552689552, 0.04519643031060696, 0.045326415672898294, 0.04749406479299068]
keep tracking of training error:
[0.058622940331697466, 0.05302516132593155, 0.04993059240281582, 0.04812738552689552, 0.04519643031060696, 0.04532

KeyboardInterrupt: 